In [2]:
import numpy as np

A schedule with T intervals can have T states. A state of a schedule at interval T is defined by $p^-_t(i)$, the amount of work left at the end of the state at interval $t-1$ and $v_t(i)$, the arriving amount of work in interval $t$.

In [125]:
"""
A schedule class with a constructor and a method to calculate the system states.
"""
class Schedule:
    def __init__(self, x, d, s):
        self.parameters = dict({'x': x, 'd': d, 's': s})
        self.system = dict({
            'p_min': np.zeros((len(self.parameters['x']), len(self.parameters['s'])), dtype=np.float64),
            'v': np.zeros((len(self.parameters['x']), len(self.parameters['s'])), dtype=np.float64)
        })
        self.system['p_min'][0][0] = 1
        self.system['v'][0] = self.parameters['s'].copy()
        self.system['p_plus'] = self.system['v'].copy()
        self.state = 0 # 0: initial state
        
    def calculate_system_states(self, until = 1):

            while self.state < until:
                """Set state to 1, because state 0 has already been calculated in the constructor."""
                self.state += 1
                
                """The probability that the amount of work left in the system equals zero just before state t starts 
                is the probablity that the total amount work in state t-1 was less than or equal to the interval length d."""
                self.system['p_min'][self.state][0] = np.sum(self.system['p_plus'][self.state-1][:self.parameters['d']])
                
                """The probability that the amount of work left in the system equals i just before state t starts 
                is the probablity that the total amount work in state t-1 exceeded the interval length d with amount i."""
                self.system['p_min'][self.state][1:(-1*self.parameters['d']+1)] = self.system['p_plus'][self.state-1][self.parameters['d']:]
                
                """Just a test. This is not the actual calculation."""
                self.system['p_plus'][self.state] = self.system['p_min'][self.state] # 
                

schedule = Schedule(x=np.array([1, 0, 2], dtype=np.float64), d=3, s=np.array(
    [0.1, 0.2, 0.3, 0.2, 0.15, 0.05], dtype=np.float64))
schedule.calculate_system_states(until = 2)
schedule.system


{'p_min': array([[1.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
        [0.6 , 0.2 , 0.15, 0.05, 0.  , 0.  ],
        [0.95, 0.05, 0.  , 0.  , 0.  , 0.  ]]),
 'v': array([[0.1 , 0.2 , 0.3 , 0.2 , 0.15, 0.05],
        [0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 0.  , 0.  , 0.  ]]),
 'p_plus': array([[0.1 , 0.2 , 0.3 , 0.2 , 0.15, 0.05],
        [0.6 , 0.2 , 0.15, 0.05, 0.  , 0.  ],
        [0.95, 0.05, 0.  , 0.  , 0.  , 0.  ]])}